# HNU CE 프로그래밍언어론 (2023년 1학기) HW4
기말과제 파트1

- 이름: 홍길동
- 학번: 99999999

이름과 학번을 위에 작성하시오.


적절한 내용과 개수의 테스트를 작성하지 않으면 감점. 테스트가 적절하지 않거나 지나치게 부족하면 함수를 올바르게 작성했더라도 점수를 얻지 못할 수 있음에 유의!

----

In [ ]:
:opt no-lint
{-# LANGUAGE ScopedTypeVariables FlexibleInstances TypeApplications RankNTypes KindSignatures #-}

----
## 문제 1. (20점) 9장 연습문제 4번

In [ ]:
type Nm = String -- 변수 이름은 문자열로
data Expr = Var Nm         -- x
          | Lam Nm Expr    -- (λx.e)
          | App Expr Expr  -- (e1 e2)
          deriving (Eq, Ord)

type Addr = Int
type Env' = [(Nm,Addr)]
data Clos' = Cl' Expr Env' deriving (Eq,Ord)

lookupEnv' :: Nm -> Env' -> [Addr]
lookupEnv' x env = case lookup x env of
                     Nothing -> []
                     Just l -> [l]

import qualified Data.HashMap.Strict as M -- M.funcName으로 활용
import Data.HashMap.Strict (HashMap, (!), (!?)) -- 앞에 M. 없이
type Store = HashMap Int Clos'

lookupStore :: Addr -> Store -> [Clos']
lookupStore l st = case st !? l of
                     Nothing -> []
                     Just cl -> [cl]
                     
type Kont' = [Either Addr (Ctx,Env')]
data Ctx = Done | App1 Expr | App2 Expr  deriving (Eq,Ord)

In [ ]:
instance Show Expr where -- LaTeX 소스코드 생성하는 Show 인스턴스 직접 선언
  showsPrec _ (Var x) = showString x
  showsPrec p (Lam x e) = showParen (p > 1) $
      showString ("\\lambda "++x++".") . showsPrec 1 e
  showsPrec p (App e1 e2) = showParen (p > 9) $
      showsPrec 9 e1 . showString "\\;" . showsPrec 10 e2
import IHaskell.Display (latex)
dTeX e = display [latex $ "$"++show e++"$"]
instance IHaskellDisplay Expr where display = dTeX

instance Show Ctx where -- LaTeX 소스코드 생성하는 Show 인스턴스 직접 선언
  showsPrec _ Done = showString "\\bullet "
  showsPrec p (App1 e2) = showParen (p > 9) $
      showString "\\bullet \\;" . showsPrec 10 e2
  showsPrec p (App2 e1) = showParen (p > 9) $
      showsPrec 9 e1 . showString "\\; \\bullet"
instance IHaskellDisplay Ctx where display = dTeX
                                               
import Data.List (intercalate)
instance {-# OVERLAPS #-} Show Env' where
  show env = "\\{"++ intercalate ",\\," (map show env) ++"\\}"
instance {-# OVERLAPS #-} Show (Nm,Addr) where
  show (x,l) = x++"{\\mapsto}"++show l
instance Show Clos' where
  show (Cl' e env) = "\\langle "++show e++","++show env++"\\rangle "
instance {-# OVERLAPS #-} Show (Addr,Clos') where
  show (l,cl) = show l++"{\\mapsto}"++show cl
instance {-# OVERLAPS #-} Show Store where
  show s = "\\{"++ intercalate ",\\," (map show $ M.toList s) ++"\\}"

instance {-# OVERLAPS #-} IHaskellDisplay Env'  where display = dTeX
instance {-# OVERLAPS #-} IHaskellDisplay (Nm,Addr) 
                                                where display = dTeX
instance IHaskellDisplay Clos'                  where display = dTeX
instance {-# OVERLAPS #-} IHaskellDisplay (Addr,Clos')
                                                where display = dTeX
instance {-# OVERLAPS #-} IHaskellDisplay Store where display = dTeX

In [ ]:
stepCESKz :: (Clos', Store, Kont') -> [(Clos', Store, Kont')]
stepCESKz (Cl' (Var x) env, st, k) = [ (cl, st, Left l : k) |
                                         l <- lookupEnv' x env,
                                         cl <- lookupStore l st ]
stepCESKz _ = undefined -- 나머지 규칙들 적절히 연습문제로 작성

In [ ]:
stepCESKz' = hat stepCESKz -- 여러 번 실행에 편리한 확장된 함수

In [ ]:
-- 테스트 작성

----
## 문제 2. (10점) 11장 연습문제 3번

교재 11장에서 필요한 정의를 가져와서 활용하라.

----
## 문제 3. (15점) 12장 연습문제 3번

모든 타입(Ty)을 원소로 나열하는 무한 리스트 `allTy`를 정의해 보라.
원소가 리스트에 나타나는 순서는 `Z`를 1개만 포하는 타입, `Z`를 2개 포함하는 타입들, `Z`를 3개 포함하는 타입들, ... 이런 순서로 나타나도록 정의하라.

그러니까 `allTy`는
$[\,\mathbb{Z},
  \;\mathbb{Z}{\to}\mathbb{Z},
  \;\mathbb{Z}{\to}\mathbb{Z}{\to}\mathbb{Z},
  \;(\mathbb{Z}{\to}\mathbb{Z}){\to}\mathbb{Z},
  \;\ldots]$와 같은 리스트이다.

In [ ]:
data Ty = Z | Arr Ty Ty  deriving (Eq,Ord)

In [ ]:
allTy = undefined -- 필요하다면 여러 개의 등식이나 다른 보조 함수를 정의해서 활용해도 좋다

In [ ]:
-- 테스트로 앞에서부터 적절한 개수의 타입들을 끊어서 보기
take 30 allTy

## 문제 4. (15점) 12장 연습문제 1번. `eval` 함수만 완성하라. `ev`는 작성하지 않아도 됨
재귀함수 관련 `undefined`로 남겨진 부분을 채워넣고 재귀함수 실행을 테스트해 보라

## 문제 5. (30점) 12장 연습문제 4번,5번,6번